# Verificación de Requisitos del Dataset

## E-Commerce Shopper Behaviour and Lifestyle

Este notebook verifica que el dataset cumpla con los siguientes requisitos:

1. **Volumen:** Mínimo 1000 registros (filas)
2. **Dimensionalidad:** Mínimo 10 columnas/variables distribuidas en:
   - Variables Numéricas (Continuas/Discretas)
   - Variables Categóricas (Nominales/Ordinales)
   - Variables Booleanas y/o Temporales (Fechas)
3. **Estado:** Debe contener imperfecciones (nulos, outliers o formatos inconsistentes)

## Importar Librerías

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

## Cargar Dataset

In [ ]:
print("Cargando dataset...")
df = pd.read_csv('e_commerce_shopper_behaviour_and_lifestyle.csv')
print(f"Dataset cargado exitosamente: {df.shape[0]:,} filas x {df.shape[1]} columnas")

## 1. Verificación de Volumen

**Requisito:** Mínimo 1,000 registros (filas)

In [ ]:
print('=' * 80)
print('1. VOLUMEN')
print('=' * 80)
print(f'Número de registros (filas): {len(df):,}')
print(f'Requisito mínimo: 1,000 registros')
cumple_volumen = len(df) >= 1000
print(f'{"✓ CUMPLE" if cumple_volumen else "✗ NO CUMPLE"}')
print('=' * 80)

## 2. Verificación de Dimensionalidad

**Requisito:** Mínimo 10 columnas/variables

In [ ]:
print('=' * 80)
print('2. DIMENSIONALIDAD')
print('=' * 80)
print(f'Número de columnas: {len(df.columns)}')
print(f'Requisito mínimo: 10 columnas')
cumple_dimensionalidad = len(df.columns) >= 10
print(f'{"✓ CUMPLE" if cumple_dimensionalidad else "✗ NO CUMPLE"}')
print('=' * 80)

### Lista de Columnas

In [ ]:
print(f'Columnas disponibles ({len(df.columns)} en total):\n')
for i, col in enumerate(df.columns, 1):
    print(f'{i:2d}. {col}')

## 3. Tipos de Variables

**Requisito:** Variables Numéricas, Categóricas, Booleanas y/o Temporales

In [ ]:
# Clasificar variables
numericas = df.select_dtypes(include=[np.number]).columns.tolist()
no_numericas = df.select_dtypes(exclude=[np.number]).columns.tolist()

# Verificar variables booleanas (binarias 0/1)
booleanas = []
for col in numericas:
    unique_vals = df[col].dropna().unique()
    if len(unique_vals) <= 2 and set(unique_vals).issubset({0, 1, 0.0, 1.0}):
        booleanas.append(col)

# Variables numéricas excluyendo booleanas
numericas_puras = [col for col in numericas if col not in booleanas]

# Verificar variables temporales
temporales = []
for col in no_numericas:
    try:
        sample = df[col].dropna().head(100)
        if len(sample) > 0:
            pd.to_datetime(sample, errors='raise')
            temporales.append(col)
    except:
        pass

# Variables categóricas (no numéricas y no temporales)
categoricas = [col for col in no_numericas if col not in temporales]

print(f'Total de variables por tipo:')
print(f'  - Numéricas: {len(numericas_puras)}')
print(f'  - Categóricas: {len(categoricas)}')
print(f'  - Booleanas: {len(booleanas)}')
print(f'  - Temporales: {len(temporales)}')

### Variables Numéricas

In [ ]:
print(f'Variables Numéricas ({len(numericas_puras)}):")
for col in numericas_puras[:10]:
    print(f'  - {col}')
if len(numericas_puras) > 10:
    print(f'  ... y {len(numericas_puras) - 10} más')

### Variables Categóricas

In [ ]:
print(f'Variables Categóricas ({len(categoricas)}):")
for col in categoricas[:10]:
    print(f'  - {col}')
if len(categoricas) > 10:
    print(f'  ... y {len(categoricas) - 10} más')

### Variables Booleanas

In [ ]:
print(f'Variables Booleanas ({len(booleanas)}):")
for col in booleanas:
    print(f'  - {col}')

### Variables Temporales (Fechas)

In [ ]:
print(f'Variables Temporales/Fechas ({len(temporales)}):")
for col in temporales:
    print(f'  - {col}')

cumple_tipos = len(numericas_puras) > 0 and len(categoricas) > 0
print(f'\n{"✓ CUMPLE" if cumple_tipos else "✗ NO CUMPLE"} - Tiene variables numéricas, categóricas, booleanas y temporales')

## 4. Estado - Imperfecciones

**Requisito:** El dataset debe contener imperfecciones (nulos, outliers o formatos inconsistentes)

### a) Valores Nulos

In [ ]:
nulos_por_columna = df.isnull().sum()
columnas_con_nulos = nulos_por_columna[nulos_por_columna > 0]
total_nulos = df.isnull().sum().sum()

if len(columnas_con_nulos) > 0:
    print('Columnas con valores nulos:')
    for col, cantidad in columnas_con_nulos.items():
        porcentaje = (cantidad / len(df)) * 100
        print(f'  - {col}: {cantidad:,} ({porcentaje:.2f}%)')
else:
    print('No se encontraron valores nulos')

print(f'\nTotal de valores nulos: {total_nulos:,}')

### b) Registros Duplicados

In [ ]:
duplicados = df.duplicated().sum()
print(f'Registros duplicados: {duplicados:,}')

### c) Estadísticas de Variables Numéricas

In [ ]:
if len(numericas_puras) > 0:
    print('Estadísticas de las primeras 5 variables numéricas:')
    display(df[numericas_puras[:5]].describe())

### d) Análisis de Variables Categóricas

In [ ]:
print('Análisis de las primeras 5 variables categóricas:\n')
for col in categoricas[:5]:
    unique_count = df[col].nunique()
    print(f'{col}: {unique_count} valores únicos')
    if unique_count <= 10:
        print(f'  Valores: {df[col].unique().tolist()}')
    print()

### e) Análisis de Variables Temporales

In [ ]:
if len(temporales) > 0:
    for col in temporales:
        print(f'{col}:')
        print(f'  Primeros valores: {df[col].head(3).tolist()}')
        try:
            dates = pd.to_datetime(df[col])
            print(f'  Rango: {dates.min()} a {dates.max()}')
        except:
            print(f'  Error al convertir a fecha')
        print()

cumple_imperfecciones = total_nulos > 0 or duplicados > 0
print(f'{"✓ CUMPLE" if cumple_imperfecciones else "✗ NO CUMPLE"} - Contiene imperfecciones para limpieza')

## Resumen Final de Cumplimiento

In [ ]:
print('=' * 80)
print('RESUMEN DE CUMPLIMIENTO')
print('=' * 80)

print(f'\n1. Volumen: {len(df):,} registros (Req: ≥1,000) - {"✓ CUMPLE" if cumple_volumen else "✗ NO CUMPLE"}')
print(f'2. Dimensionalidad: {len(df.columns)} columnas (Req: ≥10) - {"✓ CUMPLE" if cumple_dimensionalidad else "✗ NO CUMPLE"}')
print(f'   - Variables Numéricas: {len(numericas_puras)}')
print(f'   - Variables Categóricas: {len(categoricas)}')
print(f'   - Variables Booleanas: {len(booleanas)}')
print(f'   - Variables Temporales: {len(temporales)}')
print(f'3. Tipos de Variables: {"✓ CUMPLE" if cumple_tipos else "✗ NO CUMPLE"}')
print(f'4. Imperfecciones: {total_nulos:,} nulos, {duplicados:,} duplicados - {"✓ CUMPLE" if cumple_imperfecciones else "✗ NO CUMPLE"}')

print('\n' + '=' * 80)
if cumple_volumen and cumple_dimensionalidad and cumple_tipos and cumple_imperfecciones:
    print('✓✓✓ EL DATASET CUMPLE CON TODOS LOS REQUISITOS ✓✓✓')
else:
    print('✗✗✗ EL DATASET NO CUMPLE CON TODOS LOS REQUISITOS ✗✗✗')
print('=' * 80)

---

## Visualización del Dataset

In [ ]:
# Mostrar las primeras filas
print('Primeras 5 filas del dataset:')
display(df.head())

In [ ]:
# Información del dataset
print('Información del dataset:')
df.info()